## Profilowanie notebook przykład


In [ ]:
%load_ext line_profiler
%load_ext memory_profiler

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import time

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

def main():
    # Import data
    mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)

    # Create the model
    x = tf.placeholder(tf.float32, [None, 784])
    x_image = tf.reshape(x, [-1,28,28,1])

    W_conv1 = weight_variable([5, 5, 1, 1])
    b_conv1 = bias_variable([1])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    W_fc1 = weight_variable([14 * 14 * 1, 1024])
    b_fc1 = bias_variable([1024])

    h_pool1_flat = tf.reshape(h_pool1, [-1, 14*14*1])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool1_flat, W_fc1) + b_fc1)

    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([1024, 10])
    b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

    # Define loss and optimizer
    y_ = tf.placeholder(tf.float32, [None, 10])

    sess = tf.Session()
    sess.as_default()

    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
    correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    sess.run(tf.global_variables_initializer())

    time_start = time.time()

    for i in range(1000):
      batch = mnist.train.next_batch(50)
      if i%200 == 0:
        train_accuracy = accuracy.eval(session=sess, feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
      train_step.run(session=sess, feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    print(time.time() - time_start)

    print("test accuracy %g"%accuracy.eval(session=sess,feed_dict={
        x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

In [ ]:
#puszcza kilka razy i wypisuje średni czas wykonania
%timeit main()

In [ ]:
#puszcza raz i daje czasy wykonania i ilość wywołania per linia
%lprun -f main main()

In [ ]:
#puszcza raz i wypisuje peak memory
%memit main()

In [ ]:
#pokazuje ilość używanej pamięci linia po linii. chyba nie do końca to o co nam chodzi
#aby móc profilować z mprun musi być import z pliku a nie w notebooku (nie orientowałem się dlaczego tak jest)
from profiling import main
%mprun -f main main()